In [ ]:
from flask import Flask, render_template_string, request
from transformers import AutoModelForCausalLM, PreTrainedTokenizerFast
import torch
import os

app = Flask(__name__)

# === 1. 모델 준비 ===
model_name = "skt/kogpt2-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

log_file = "chat_log.txt"  # 대화 기록 저장 파일

# === 2. 이전 대화 기록 불러오기 ===
chat_history = ""
if os.path.exists(log_file):
    with open(log_file, "r", encoding="utf-8") as f:
        chat_history = f.read()

# === 3. 웹 UI 템플릿 ===
page_template = """
<!DOCTYPE html>
<html>
<head>
    <title>한국어 GPT-2 챗봇</title>
</head>
<body style="font-family: Arial; margin: 20px;">
    <h2>한국어 GPT-2 대화형 챗봇</h2>
    <div style="border:1px solid #ccc; padding:10px; width:500px; height:300px; overflow-y:scroll; white-space:pre-line;">
        {{ chat_history }}
    </div>
    <form method="post">
        <input name="message" placeholder="메시지를 입력하세요" style="width:400px;">
        <input type="submit" value="Send">
    </form>
</body>
</html>
"""

# === 4. 라우팅 ===
@app.route("/", methods=["GET", "POST"])
def index():
    global chat_history
    if request.method == "POST":
        user_input = request.form["message"]

        # 대화 기록 업데이트
        chat_history += f"User: {user_input}\nBot:"

        # 최근 대화 일부만 사용 (길이 제한)
        max_history_lines = 5  # 최근 5개 대화
        history_lines = chat_history.strip().split("\n")
        recent_history = "\n".join(history_lines[-max_history_lines*2:])

        # 토큰화 및 device로 이동
        inputs = tokenizer(recent_history, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # 응답 생성 (max_new_tokens 사용)
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,       # 새로 생성할 토큰 수
            do_sample=True,
            top_p=0.9,
            temperature=0.8,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

        # 토큰 디코딩
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # AI 모델의 출력 텐서(outputs)를 사람이 읽을 수 있는 문자열로 변환(디코딩)
        # skip_special_tokens=True: <bos>, <eos>, <pad>와 같은 모델 훈련에 사용되는 특수 토큰들을 최종 문자열에서 제거하여 깨끗한 텍스트를 얻음
        bot_reply = response.split("Bot:")[-1].split("User:")[0].strip()
        # 디코딩된 전체 응답 문자열에서 챗봇이 생성한 답변 부분만 정확하게 추출
        chat_history += f" {bot_reply}\n"
        # 문자열을 구분자 "Bot:"을 기준으로 나눔
        # "User: 안녕\nBot: 안녕하세요\nUser:..."에서 ["User: 안녕\n", " 안녕하세요\nUser:..."]
        # 파일에 저장
        
        with open(log_file, "w", encoding="utf-8") as f:
            f.write(chat_history) # 로그파일에 저장해ㅔ서 파음에 열때도 사

    return render_template_string(page_template, chat_history=chat_history)

# === 5. 실행 ===
if __name__ == "__main__":
    app.run(debug=True)
